In [24]:
import pandas as pd
import numpy as np
Billing = pd.read_csv("FinalCSVs/Billing.csv")
DiseaseCase = pd.read_csv("FinalCSVs/DiseaseCase.csv")

/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# Compare # of unique DM2 patients in Billing and DiseaseCase
diabetes = ["Diabetes Mellitus (ml)","Diabetes Mellitus"]
DC_dm2 = DiseaseCase.loc[DiseaseCase['Disease'].isin(diabetes)]
DC_dm2_id = DC_dm2['Patient_ID'].unique()

Billing2 = Billing.dropna(subset=['DiagnosisCode_calc']) # Remove NA
Billing2 = Billing2[pd.to_numeric(Billing2.DiagnosisCode_calc,errors='coerce').notnull()] # convert diagnosis codes to floats
Billing2['DiagnosisCode_calc'] = pd.to_numeric(Billing2['DiagnosisCode_calc']) # convert diagnosis codes to floats
BL_dm2 = Billing2.loc[(Billing2['DiagnosisCode_calc'] >= 250.0) & (Billing2['DiagnosisCode_calc'] < 251.0)] #Extract DM2 diagnosis patients into new dataframe
BL_dm2_id = BL_dm2['Patient_ID'].unique()

print(len(DC_dm2_id), len(BL_dm2_id)) #print number of unique patients w/ DM2
# The unique list of DM2 patients_IDs are stored in the DC_dm2_id variable


20916 3380


In [26]:
uniqueDM2Diag = BL_dm2.drop_duplicates(subset = ['Patient_ID','DiagnosisCode_calc'])
uniqueDM2Diag['DiagnosisCode_calc'].value_counts()
# 250.00 = DM2
# 250.02 = DM2 w/ without complication
# 250.93 = DM1 w/ unspecified condition [juvenile type]
# 250.01 = DM1 w/ without complication [juvenile type]
# 250.70 = DM2 w/ PCD (PVD)
# 250.10 = DM2 w/ Ketoacidosis
# 250.60 = DM2 w/ neurological manifestations (neuropathy?)

250.00    1645
250.02    1562
250.93     480
250.01      64
250.70       9
250.10       5
250.60       1
Name: DiagnosisCode_calc, dtype: int64

In [27]:
# Complications in our Diabetes population
DM2BillingSet = Billing2.loc[Billing2['Patient_ID'].isin(DC_dm2_id)]
print(len(DM2BillingSet.index))

44675


In [28]:
DM2BillingSet.head()

,Billing_ID,Network_ID,Site_ID,Patient_ID,Encounter_ID,ServiceDate,ServiceCode,DiagnosisText_orig,DiagnosisText_calc,DiagnosisCodeType_orig,DiagnosisCodeType_calc,DiagnosisCode_orig,DiagnosisCode_calc,DateCreated
531136,1103465,8,8,80080109810,NaN,2016-11-21,8513,NaN,"INFECTION BY HISTOPLASMA DUBOISII, WITHOUT MEN...",ICD9,ICD9,11510,115.10,2016-11-21
1010470,2864826,8,25,80250030745,NaN,2013-09-19,8005,ARTHROT/PROS REMOV-HIP,OSTEOARTHROSIS AND ALLIED DISORDERS,NaN,ICD9,NaN,715.00,2013-09-19
1010471,2864827,8,25,80250030745,NaN,2013-09-18,8005,ARTHROT/PROS REMOV-HIP,OSTEOARTHROSIS AND ALLIED DISORDERS,NaN,ICD9,NaN,715.00,2013-09-18
1458507,5713516,8,1,80013241698,7172841.0,2019-02-11,78513,NaN,OTHER MALIGNANT LYMPHOMAS INVOLVING INTRA-ABDO...,ICD9,ICD9,20283,202.83,2019-02-11
1458508,5713517,8,1,80013241698,7738032.0,2019-02-26,78513,NaN,UNSPECIFIED ESSENTIAL HYPERTENSION,ICD9,ICD9,4019,401.90,2019-02-26


In [55]:
## The 10 Complications
DM2 = [float(item/100) for item in range(25000,25100)]
AP = [float(item/100) for item in range(41300,41400)] # http://www.icd9data.com/2013/Volume1/390-459/410-414/413/default.htm
AS = [float(item/100) for item in range(44000,44100)] # http://www.icd9data.com/2012/Volume1/390-459/440-449/440/default.htm
ICHD = [float(item/100) for item in range(41400,41500)] # http://www.icd9data.com/2015/Volume1/390-459/410-414/414/default.htm
DD = [311] # http://www.icd9data.com/2012/Volume1/290-319/300-316/311/default.htm
DNP = [float(item/100) for item in range(58500,58600)]
DNP.append(581.81) # include 250.4? http://www.icd9data.com/2012/Volume1/580-629/580-589/585/default.htm
DNU = [float(item/100) for item in range(35700,35800)] # include 250.6? 357.2 is the exact code http://www.icd9data.com/2012/Volume1/320-389/350-359/357/default.htm 
DR = [float(item/100) for item in range(36200,36210)] # http://www.icd9data.com/2015/Volume1/320-389/360-379/362/default.htm
HL = [389.9] #http://www.icd9data.com/2012/Volume1/320-389/380-389/389/default.htm
MI = [float(item/100) for item in range(41000,41100)] #http://www.icd9data.com/2014/Volume1/390-459/410-414/410/default.htm
PVD = [float(item/100) for item in range(43300,43400)] #http://www.icd9data.com/2013/Volume1/390-459/440-449/443/default.htm
complications = AP + AS + ICHD + DD + DNU + DNP +DR + HL + MI + PVD + DM2

In [56]:
complicationDF = DM2BillingSet.loc[DM2BillingSet['DiagnosisCode_calc'].isin(complications)]
complicationDF = complicationDF.drop_duplicates(subset = ['Patient_ID', 'DiagnosisCode_calc'])

In [57]:
complicationDF['DiagnosisCode_calc'].value_counts()

250.00    1634
250.02    1557
250.93     480
311.00     334
414.00     305
250.01      63
414.90      54
585.90      47
357.90      26
413.90      22
389.90      15
250.70       9
410.90       7
440.90       6
440.21       5
250.10       4
585.60       4
433.10       3
414.80       2
250.60       1
Name: DiagnosisCode_calc, dtype: int64

In [64]:
print(sum(complicationDF['Patient_ID'].value_counts() == 4)) # 15 patients have 4 complications 
print(sum(complicationDF['Patient_ID'].value_counts() == 3)) # 85 patients have 3 complications
print(sum(complicationDF['Patient_ID'].value_counts() == 2)) # 724 pateints have 2 complications
print(sum(complicationDF['Patient_ID'].value_counts() == 1)) # 2815 have just 1 complication
#print(complicationDF['Patient_ID'].value_counts() == 4)

15
85
724
2815


In [67]:
# Example patient with visits for each complication diagnosis
complicationDF[complicationDF['Patient_ID'] == 80613312481]

,Billing_ID,Network_ID,Site_ID,Patient_ID,Encounter_ID,ServiceDate,ServiceCode,DiagnosisText_orig,DiagnosisText_calc,DiagnosisCodeType_orig,DiagnosisCodeType_calc,DiagnosisCode_orig,DiagnosisCode_calc,DateCreated
1812884,5769772,8,61,80613312481,7526395.0,2019-02-11,78005,NaN,TYPE II DIABETES MELLITUS [NON-INSULIN DEPENDE...,ICD9,ICD9,25002,250.02,2019-02-11
1812885,5769773,8,61,80613312481,7533012.0,2019-03-01,78529,NaN,DIABETES MELLITUS TYPE I [INSULIN DEPENDENT TY...,ICD9,ICD9,25093,250.93,2019-03-01
1812888,5769776,8,61,80613312481,7840247.0,2019-03-26,78005,NaN,CORONARY ATHEROSCLEROSIS OF UNSPECIFIED TYPE O...,ICD9,ICD9,41400,414.00,2019-04-01
1812916,5769779,8,61,80613312481,7058504.0,2019-06-19,78529,NaN,"CHRONIC KIDNEY DISEASE, UNSPECIFIED",ICD9,ICD9,5859,585.90,2019-06-19
